In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os, sys
sys.path.append('/home/hrai/codes/hpe_library/')
from lib_import import *
from my_utils import *
os.chdir('/home/hrai/codes/MotionBERT')

In [2]:
def get_canonical_3d_fixed_dist(world_3d, C):
    num_frames = len(data['world_3d'])
    canonical_3d = world_3d.copy()
    cam_origin = C.copy()
    pelvis = world_3d[:, 0].copy()
    vec_cam_origin_to_pelvis = pelvis - cam_origin
    mag_cam_origin_to_pelvis = np.expand_dims(np.linalg.norm(vec_cam_origin_to_pelvis, axis=1), axis=1).repeat(3, axis=1) # (F, 3)
    vec_cam_forward = np.multiply(np.expand_dims(R[2], 0).repeat(num_frames, axis=0),  mag_cam_origin_to_pelvis)
    canonical_pelvis = cam_origin + vec_cam_forward
    canonical_3d = canonical_3d - np.expand_dims(pelvis, 1) + np.expand_dims(canonical_pelvis, 1)
    return canonical_3d

In [3]:
h36m_3d_world, h36m_cam_param = load_h36m()

==> Loading 3D data wrt World CS...


In [4]:
save_folder = '/home/hrai/codes/MotionBERT/data/motion3d/h36m_gt_pkl'
if not os.path.exists(save_folder): os.makedirs(save_folder)

### canonical_3d, joint_2d_from_canonical_3d (same dist)

In [10]:
for item in tqdm(glob(save_folder + '/*.pkl')):
    data = readpkl(item)
    if ('canonical_3d' in data) and ('joint_2d_from_canonical_3d' in data): continue
    cam_param = data['cam_param'][0].reshape(1)[0].copy()
    int_mat = cam_param['intrinsic']
    ext_mat = cam_param['extrinsic']
    cam_proj = int_mat @ ext_mat
    R = ext_mat[:3, :3]
    t = ext_mat[:3, 3]
    C = T_to_C(R, t)
    world_3d = np.array(data['world_3d']).copy() # (F, J, 3)
    
    canonical_3d = get_canonical_3d_fixed_dist(world_3d, C)
    pose_2d_from_3d_canonical = projection(canonical_3d, cam_proj)[..., :2]
    
    data['canonical_3d'] = canonical_3d.tolist()
    data['joint_2d_from_canonical_3d'] = pose_2d_from_3d_canonical.tolist()
    
    savepkl(data, item)
    #break

  1%|▏         | 12/836 [00:02<02:05,  6.57it/s]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x775c983c70a0>>
Traceback (most recent call last):
  File "/home/hrai/miniconda3/envs/motionbert/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
  8%|▊         | 68/836 [00:13<02:14,  5.70it/s]

### canonical_3d_fixed, joint_2d_from_canonical_3d_fixed (fixed dist)

In [17]:
load_folder = '/home/hrai/codes/MotionBERT/data/motion3d/h36m_gt_pkl_canonical_3d_same_dist'
save_folder = '/home/hrai/codes/MotionBERT/data/motion3d/h36m_gt_pkl_canonical_3d_fixed_dist'
if not os.path.exists(save_folder): os.makedirs(save_folder)

In [21]:
for item in glob(load_folder + '/*.pkl'):
    print(item)
    data = readpkl(item)
    cam_param = data['cam_param'][0].reshape(1)[0].copy()
    int_mat = cam_param['intrinsic']
    ext_mat = cam_param['extrinsic']
    cam_proj = int_mat @ ext_mat
    R = ext_mat[:3, :3]
    t = ext_mat[:3, 3]
    C = T_to_C(R, t)
    
    canonical_3d_same_dist = np.array(data['canonical_3d'].copy())
    #pose_2d_from_3d_canonical = projection(canonical_3d, cam_proj)[..., :2]
    
    #data['canonical_3d_fixed'] = canonical_3d.tolist()
    #data['joint_2d_from_canonical_3d_fixed'] = pose_2d_from_3d_canonical.tolist()
    break

/home/hrai/codes/MotionBERT/data/motion3d/h36m_gt_pkl_canonical_3d_same_dist/S1_54138969_Walking.pkl


In [22]:
canonical_3d_same_dist.shape

(3134, 17, 3)

In [23]:
canonical_3d_same_dist[0]

array([[-0.27945777,  0.18764733,  0.57665961],
       [-0.41184818,  0.17699148,  0.58253889],
       [-0.38027463,  0.24077074,  0.14539932],
       [-0.35607179,  0.32021284, -0.30115039],
       [-0.14706715,  0.19830319,  0.57078032],
       [-0.1599361 ,  0.23709846,  0.12977605],
       [-0.15630869,  0.31454275, -0.31776483],
       [-0.29667907,  0.22135777,  0.80704068],
       [-0.30477095,  0.23137896,  1.06379544],
       [-0.31830626,  0.17265349,  1.16887532],
       [-0.31585147,  0.27000086,  1.2300552 ],
       [-0.17279775,  0.23192067,  0.99035071],
       [-0.08189077,  0.21475497,  0.72725974],
       [-0.01587575,  0.14900417,  0.49340384],
       [-0.43060636,  0.23386096,  0.98031019],
       [-0.49696221,  0.2381245 ,  0.70945953],
       [-0.53859799,  0.17745735,  0.46872472]])

In [25]:
data.keys()

dict_keys(['joint_2d', 'confidence', 'joint3d_image', 'joints_2.5d_image', '2.5d_factor', 'camera_name', 'action', 'source', 'frame', 'world_3d', 'cam_3d', 'cam_param', 'canonical_3d', 'joint_2d_from_canonical_3d'])